함수형 프로그래밍 패러다임에 있어 매우 중요한 특징 하나는 고차 함수다. 고차 함수는 다른 함수를 인자로 받거나 함수를 결과로 반환하는 함수다. 파이썬은 여러 종류의 고차 함수를 지원한다는 것에 대해 살펴보고, 그러한 함수를 논리적으로 확장한 것을 다룰 것이다. 앞으로 살펴보겠지만, 고차 함수에는 다음과 같이 세 가지 유혀잉 존재한다.

* 인자 중 하나로 함수를 요구하는 함수
* 함수를 반환하는 함수
* 함수를 인자로 받고, 함수를 반환하는 함수

함수를 만들어 내는 함수라는 개념은 조금 이상하게 들릴수도 있다. 하지만 Callable 클래스의 객체에 대해 공부한다면 호출 가능한 객체를 반환하는 함수를 보게 된다. 그러한 함수는 다른 함수를 만들어 내는 함수의 한 가지 예다.

함수를 인자로 받아 다른 함수를 만들어 내는 함수의 예로는 복잡한 Callable 클래스나 함수 데코레이터가 있다.

앞에서 설명한 여러 컬렉션 함수의 고차 함수 버전이 있었으면 할 때가 있다. 이번 장에서는 커다란 튜플에서 특정 필드를 추출하여 축약을 수행하는 출약(추출()) 설계 패턴에 대해 살펴본다. 또한 이러한 공통 컬렉션 처리 함수를 직접 정의하는 방법에 대해 살펴본다.

이번 장에서는 다음과 같은 함수를 살펴본다.

* max()와 min()
* 고차 함수를 호출할 때 편하게 쓸 수 있는 람다식
* map()
* filter()
* iter()
* sorted()

max()와 min() 함수는 축약이다. 컬렉션으로부터 값을 하나 만들어 낸다. 다른 함수들은 매핑이다. 이들은 하나의 값으로 축약시키지 않고 컬렉션을 변환한다.

### max()와 min()을 사용해 최댓값, 최솟값 알아내기

max()와 min() 함수는 두 가지 동작이 가능하다. 이들은 컬렉션에 적용할 수 있는 단순한 함수지만, 또한 고차 함수이기도 하다. 이들의 기본적인 동작은 다음과 같다.

In [1]:
max(1,2,3)

3

In [2]:
max((1,2,3))

3

두 함수 모두 임의의 길이 인자를 받는다. 이들은 유일한 인자로 시퀀스뿐만 아니라 반복 가능 객체도 받도록 설계됐고, 인자로 받은 반복 가능 객체의 원소 중 최댓값을 찾는다.

좀 더 복잡한 동작도 가능하다. 4 장에서 본 여행 경로 데이터를 생각해보자. 거기서 다음과 같은 튜플의 시퀀스를 만들어 내는 함수를 살펴봤다. 

In [7]:
trip = (((37.54, -76.33), (37.84, -76.27), 17.72), ((37.84, -76.27), (38.33, -76.45), 30.73), ((38.33, -76.45), (38.84, -76.53), 31.07))

각 3-튜플에는 시작 지점의 위치, 끝 지점의 위치 거리 값이 들어 있다. 위치는 위도와 경도의 쌍이다. 이 시퀀스에서 최대와 최소 거리를 찾는 데에는 세 가지 방법이 있다. 각각은 다음과 같다.

* 거리는 제너레이터 함수로 뽑아낸다. 그 함수는 오직 거리만을 돌려주고, 해당 구간의 시작과 끝 위치는 버린다. 최댓값이나 최솟값 구간의 위치 정보를 가지고 추가 처리를 해야 하는 경우에는 이러한 방식이 잘 들어맞지 않을 것이다. 

* 풀기(처리(감싸기())) 패턴을 사용한다. 이렇게 하면, 최대, 최소 거리가 있는 구간을 얻을 수 있다. 그로부터 거리만 필요하다면 거리만을 뽑아낼 수도 있다. 구간을 나타내는 두 위치 정보를 사용해 어떤 구간이 최대 또는 최소인지를 파악할 수 있따.

* max()와 min()을 고차 함수로 사용한다.


다음은 풀기(처리(감싸기())) 패턴을 사용한 것이다. 실제로토 wrap()와 unwrap() 이라는 이름의 함수를 정의하여 패턴이 어떻게 동작하는지 명확히 이해할 수 있게 했다.

In [8]:
def wrap(leg_iter):
    return ((leg[2], leg) for leg in leg_iter)

def unwrap(dist_leg):
    distance, leg = dist_leg
    return leg

long, short = unwrap(max(wrap(trip))), unwrap(min(wrap(trip)))
print(long, short)

((38.33, -76.45), (38.84, -76.53), 31.07) ((37.54, -76.33), (37.84, -76.27), 17.72)


마지막으로 가장 중요한 형태는 max()와 min() 고차 함수 기능을 사용한다. 이를 위해 먼저 도우미 함수를 정의하고 그 함수를 사용하여 구간의 컬렉션을 원하는 요약 형태로 축약한다. 다음을 살펴보자.

In [9]:
def by_dist(leg):
    lat, lon, dist = leg
    return dist
long, short = max(trip, key=by_dist), min(trip, key=by_dist)
print(long, short)

((38.33, -76.45), (38.84, -76.53), 31.07) ((37.54, -76.33), (37.84, -76.27), 17.72)


by_dist() 함수는 구간 leg 튜플의 세 원소를 분리하여 거리를 반환한다. 이를 max(0와 min() 함수에 사용한다.

max()와 min() 함수는 모두 반복 가능 챛ㅇ게와 함수를 인자로 받는다. 파이썬의 모든 고차 함수에서는 키워드 매개변수 key=를 사용해 필요한 키 값을 뽑아내도록 할 수 있다.

max() 함수가 key로 지정한 함수를 어떻게 사용하는지 다음 코드가 개념화해 보여준다.

In [14]:
wrap = ((key(leg), leg) for leg in trip)

### 파이썬의 람다 식 사용하기

많은 경우 도우미 함수를 따로 정의하는 것은 필요 이상의 코드를 작성하게 된다. 